In [1]:
import pandas 

kinetics = pandas.read_csv( 'kinetics.csv' )
thermal = pandas.read_csv( 'thermal.csv' ) 

print( kinetics.columns )
print( thermal.columns ) 

Index(['well', 'rate', 'mutant', 'protein_yield', 'dilution', 'note'], dtype='object')
Index(['well', 'rate', 'mutant'], dtype='object')


In [2]:
# for thermal 
# convert well indexes into temperatures based on map 

temp_key = pandas.read_csv( 'temperature_key.csv', index_col='Well' )
thermal['temp'] = thermal.well.str[0].map( temp_key.Celsius.to_dict() ) 

print( thermal.sample( 3 ) )

     well      rate   mutant  temp
2143   G4  0.000288    W325A  32.9
3152   G3  0.000026  H119N_D  32.9
2616   A9  0.000024    W409Y  50.0


In [3]:
# for kinetics, convert the rate into units (1/min) 
# hard-coded values:
#  0.0002 is the conversion from OD to M
#  0.25 is because we dilute the protein 4-fold upon initiation 
#  113330 is the extinction coef of BglB, used to convert mg/mL to M

kinetics[ 'kobs' ] = kinetics['rate'] * 0.0002 / ( kinetics[ 'protein_yield' ] * kinetics[ 'dilution' ] * 0.25 / 113300 )

print( kinetics.sample( 3 ) ) 

    well      rate mutant  protein_yield  dilution  note       kobs
169  B10  0.018800  N220G           2.18      0.01   NaN  78.166606
437   F7  0.000029  W399R           2.13      0.10   NaN   0.012468
259   D9  0.000015  E164G           0.21      0.10   NaN   0.063016


In [4]:
# also for kinetics, convert well indexes into substrate concentrations

def my_s( x ):
    my_ss = dict( zip( 'ABCDEFGH', [ .075, .01875, .0046875, .001171875, .0002929688, .0000732422, .0000183105, 0 ] ) )
    return my_ss[ x[0] ] 

kinetics[ 's' ] = kinetics.well.map( my_s ) 

In [5]:
# anything we need to manually drop? 

to_drop = [ 'WT', 'WT1' ] 

for df in [ kinetics, thermal ]:
    for item in to_drop:
        if item in df.index:
            df.drop( item, inplace=True ) 

In [6]:
# now export "cleaned" data 

kinetics.to_csv( '../clean/kinetics.csv' ) 
thermal.to_csv( '../clean/thermal.csv' ) 